In [11]:
# Zelle 1: Setup & Config
from google.cloud import bigquery
import pandas as pd
import logging

# Config
PROJECT_ID = "taxi-bi-project"  
DIM_DATASET = "dimensional"     # Quelle (Star Schema)
AGG_DATASET = "aggregational"       # Ziel (Data Marts)

client = bigquery.Client(project=PROJECT_ID)

# Hilfsfunktion, um Datasets zu finden/erstellen
def create_dataset_if_not_exists(dataset_id):
    full_dataset_id = f"{PROJECT_ID}.{dataset_id}"
    try:
        client.get_dataset(full_dataset_id)
        print(f"✅ Ziel-Dataset gefunden: {full_dataset_id}")
    except:
        print(f"Erstelle neues Dataset: {full_dataset_id} ...")
        # Wir holen uns die Region vom Quell-Dataset, damit alles gleich liegt (EU/US)
        src_ds = client.get_dataset(f"{PROJECT_ID}.{DIM_DATASET}")
        new_ds = bigquery.Dataset(full_dataset_id)
        new_ds.location = src_ds.location
        client.create_dataset(new_ds)
        print(f"✅ Dataset erstellt (Region: {src_ds.location})")

create_dataset_if_not_exists(AGG_DATASET)

/Users/vincentgoldmann/Documents/GitHub/bi_project_task2/venv/lib/python3.13/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Ziel-Dataset gefunden: taxi-bi-project.aggregational


In [8]:
# Zelle 2: Aggregation 1 - Monthly KPIs (Management View)
# Diese Tabelle beantwortet: "Wie entwickeln sich Umsatz und Fahrtenzahlen?"

def create_monthly_kpis():
    print("--- 1. Erstelle Tabelle: agg_monthly_kpis ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis`
    AS
    SELECT
        -- Dimensionen (Woran wollen wir schneiden?)
        d.year,
        d.month,
        d.month_name,
        d.quarter,
        f.source_system,      -- Yellow vs Green vs FHV
        v.vendor_name,        -- Creative Mobile vs Uber/Lyft Bases
        p.payment_description, -- Cash vs Credit
        
        -- Metriken (Hier wird gerechnet!)
        COUNT(f.trip_id) AS total_trips,
        
        -- Summen (Runden auf 2 Nachkommastellen spart Speicher und sieht besser aus)
        ROUND(SUM(f.total_amount), 2) AS total_revenue,
        ROUND(SUM(f.fare_amount), 2) AS total_fare,
        ROUND(SUM(f.tip_amount), 2) AS total_tips,
        
        -- Durchschnitte (KPIs)
        ROUND(AVG(f.total_amount), 2) AS avg_ticket_size,
        ROUND(AVG(f.trip_distance), 2) AS avg_distance_miles,
        ROUND(AVG(f.duration_minutes), 1) AS avg_duration_min

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- JOINs zum Star Schema
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_vendor` v ON f.vendor_id = v.vendor_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_payment_type` p ON f.payment_type_id = p.payment_type_id

    -- FILTER:
    -- Wir wollen hier nur "echte" Fahrten für die Statistik.
    -- Wir schließen 0$-Fahrten (Geister) aus.
    -- Aber: Deine 3.80$ Fahrt (Short Trip) bleibt drin, weil total_amount > 0!
    WHERE f.total_amount > 0
    
    GROUP BY 1, 2, 3, 4, 5, 6, 7
    ORDER BY year DESC, month DESC, total_revenue DESC
    """
    
    try:
        job = client.query(sql)
        job.result() # Warten auf Fertigstellung
        print("✅ agg_monthly_kpis erfolgreich erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_monthly_kpis()

--- 1. Erstelle Tabelle: agg_monthly_kpis ---
✅ agg_monthly_kpis erfolgreich erstellt.


In [ ]:
# Zelle 3: Aggregation 2 - Geo Stats (Map View)
# Diese Tabelle beantwortet: "Welche Stadtteile sind am lukrativsten?"

def create_geo_stats():
    print("--- 2. Erstelle Tabelle: agg_geo_stats ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats`
    AS
    SELECT
        d.year,
        d.quarter,
        
        -- Feldnamen auf Kleinschreibung korrigiert
        loc.borough AS pickup_borough,
        loc.zone AS pickup_zone,
        loc.service_zone,
        
        f.source_system,
        
        COUNT(f.trip_id) AS pickup_count,
        ROUND(SUM(f.total_amount), 0) AS total_revenue_generated,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip_here

    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- Korrektur: location_id statt LocationID
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc ON f.pickup_location_id = loc.location_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key

    WHERE f.total_amount > 0
    
    GROUP BY 1, 2, 3, 4, 5, 6
    ORDER BY pickup_count DESC
    """
    
    try:
        job = client.query(sql)
        job.result()
        print("✅ agg_geo_stats erfolgreich erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_geo_stats()

--- 2. Erstelle Tabelle: agg_geo_stats ---
✅ agg_geo_stats erfolgreich erstellt.


In [19]:
# Zelle 4: Quality Check (Kurzer Blick auf das Ergebnis)
# Prüfen, ob die Tabellen gefüllt sind und die Geisterfahrten weg sind

def check_aggregation():
    print("\n--- CHECK: Monthly KPIs (Top 5 Rows) ---")
    query_kpi = f"""
    SELECT year, month_name, source_system, total_trips, total_revenue, avg_ticket_size 
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_monthly_kpis` 
    ORDER BY total_trips DESC 
    LIMIT 5
    """
    print(client.query(query_kpi).to_dataframe().to_string(index=False))

    print("\n--- CHECK: Geo Stats (Top 5 Zones) ---")
    query_geo = f"""
    SELECT pickup_borough, pickup_zone, source_system, pickup_count, total_revenue_generated
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_geo_stats`
    WHERE pickup_zone != 'NV' -- Wir ignorieren kurz die Unknowns
    ORDER BY pickup_count DESC
    LIMIT 5
    """
    print(client.query(query_geo).to_dataframe().to_string(index=False))

check_aggregation()


--- CHECK: Monthly KPIs (Top 5 Rows) ---


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 year month_name source_system  total_trips  total_revenue  avg_ticket_size
 <NA>       None        YELLOW     18583128   212632111.82            11.44
 <NA>       None        YELLOW     18571532   207398754.56            11.17
 <NA>       None        YELLOW     17350741   278040333.89            16.02
 <NA>       None        YELLOW     16939149   267008644.23            15.76
 2015       June        YELLOW      4056201    73857611.68            18.21

--- CHECK: Geo Stats (Top 5 Zones) ---
pickup_borough               pickup_zone source_system  pickup_count  total_revenue_generated
     Manhattan     Upper East Side South        YELLOW       2577047               27047460.0
     Manhattan            Midtown Center        YELLOW       2472275               30124537.0
     Manhattan              East Village        YELLOW       2390112               29673556.0
     Manhattan Times Sq/Theatre District        YELLOW       2374174               31971137.0
     Manhattan              Midtow

/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [22]:
def create_time_trends():
    print("--- 3. Erstelle Tabelle: agg_time_trends ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `taxi-bi-project.dimensional.dim_date` AS
    SELECT
        datum AS date_key,
        EXTRACT(YEAR FROM datum) AS year,
        EXTRACT(MONTH FROM datum) AS month,
        FORMAT_DATE('%B', datum) AS month_name,
        FORMAT_DATE('%A', datum) AS day_name,             -- WICHTIG: Für dein Skript
        EXTRACT(DAYOFWEEK FROM datum) AS day_of_week_num, -- WICHTIG: Für dein Skript
        EXTRACT(QUARTER FROM datum) AS quarter,
        CASE WHEN EXTRACT(DAYOFWEEK FROM datum) IN (1, 7) THEN TRUE ELSE FALSE END AS is_weekend
    FROM UNNEST(GENERATE_DATE_ARRAY('2010-01-01', '2025-12-31')) AS datum;
    """
    try:
        client.query(sql).result()
        print("✅ agg_time_trends erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_time_trends()

--- 3. Erstelle Tabelle: agg_time_trends ---
✅ agg_time_trends erstellt.


In [12]:
def create_route_stats():
    print("--- 4. Erstelle Tabelle: agg_route_stats ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats` AS
    SELECT
        d.year,
        
        -- VON -> NACH
        pu.Borough AS pickup_borough,
        do.Borough AS dropoff_borough,
        
        -- Optional: Wenn du es feiner willst, nimm die Zonen dazu (wird aber größer)
        -- pu.Zone as pickup_zone,
        -- do.Zone as dropoff_zone,
        
        COUNT(*) AS trip_count,
        ROUND(AVG(f.total_amount), 2) AS avg_cost,
        ROUND(AVG(f.duration_minutes), 1) AS avg_duration
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` pu ON f.pickup_location_id = pu.LocationID
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` do ON f.dropoff_location_id = do.LocationID
    
    WHERE f.total_amount > 0 
      AND pu.Borough != 'Unknown' 
      AND do.Borough != 'Unknown'
      
    GROUP BY 1, 2, 3
    ORDER BY trip_count DESC
    """
    try:
        client.query(sql).result()
        print("✅ agg_route_stats erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_route_stats()

--- 4. Erstelle Tabelle: agg_route_stats ---
❌ Fehler: 400 Name LocationID not found inside pu at [20:85]; reason: invalidQuery, location: query, message: Name LocationID not found inside pu at [20:85]

Location: EU
Job ID: 9507c2d4-584c-44a9-b219-90b31158dc8c



In [13]:
def create_airport_stats():
    print("--- 5. Erstelle Tabelle: agg_airport_trips ---")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips` AS
    SELECT
        d.year,
        d.month_name,
        f.source_system,
        
        -- War es eine Fahrt ZUM oder VOM Flughafen?
        CASE 
            WHEN rc.rate_description LIKE '%JFK%' OR rc.rate_description LIKE '%Newark%' THEN 'Airport Rate'
            ELSE 'Standard Rate to Airport Zone'
        END AS trip_category,
        
        COUNT(*) AS total_trips,
        ROUND(AVG(f.total_amount), 2) AS avg_ticket,
        ROUND(AVG(f.tip_amount), 2) AS avg_tip
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_rate_code` rc ON f.rate_code_id = rc.rate_code_id
    
    -- Location Filter: 132=JFK, 138=LaGuardia, 1=Newark
    WHERE (f.pickup_location_id IN (132, 138, 1) OR f.dropoff_location_id IN (132, 138, 1))
      AND f.total_amount > 0
      
    GROUP BY 1, 2, 3, 4
    """
    try:
        client.query(sql).result()
        print("✅ agg_airport_trips erstellt.")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_airport_stats()

--- 5. Erstelle Tabelle: agg_airport_trips ---
✅ agg_airport_trips erstellt.


In [14]:
def create_quality_audit_mart():
    print("--- 8. Erstelle Tabelle: agg_quality_audit ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_quality_audit` AS
    SELECT
        DATE_TRUNC(pickup_datetime, MONTH) as month,
        source_system,
        COUNT(*) as total_trips,
        COUNTIF(trip_distance = 0 AND source_system != 'FHV') as gps_failures,
        COUNTIF(pickup_location_id IN (263, 264)) as unknown_locations,
        COUNTIF(dq_issue_flag = TRUE) as total_issues
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips`
    GROUP BY 1, 2
    """
    client.query(sql).result()
    print("✅ agg_quality_audit erstellt.")

In [15]:
def create_shared_ride_stats():
    print("--- 7. Erstelle Tabelle: agg_shared_rides ---")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{AGG_DATASET}.agg_shared_rides` AS
    SELECT
        d.year,
        f.source_system,
        f.sr_flag,
        COUNT(*) AS trip_count,
        ROUND(AVG(f.fare_amount), 2) AS avg_fare -- Nur für Yellow/Green sinnvoll
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d ON f.pickup_date_key = d.date_key
    GROUP BY 1, 2, 3
    """
    client.query(sql).result()
    print("✅ agg_shared_rides erstellt.")

In [16]:
def check_new_marts():
    # Stelle sicher, dass die Variablen passen
    PROJECT_ID = "taxi-bi-project"
    AGG_DATASET = "aggregational"
    
    print("--- 📊 FINAL DATA MART CHECK ---")

    # 1. RUSH HOUR (Wann ist am meisten los?)
    print("\n🕐 TOP 5 ZEITFENSTER (agg_time_trends):")
    # Wir sortieren nach trip_count, um die geschäftigsten Stunden zu sehen
    sql_time = f"""
    SELECT day_name, hour_of_day, source_system, trip_count, avg_fare
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_time_trends`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_time = client.query(sql_time).to_dataframe()
        print(df_time.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 2. ROUTEN (Wer fährt von wo nach wo?)
    print("\n📍 TOP 5 ROUTEN (agg_route_stats):")
    sql_routes = f"""
    SELECT pickup_borough, dropoff_borough, trip_count, avg_cost
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_route_stats`
    ORDER BY trip_count DESC
    LIMIT 5
    """
    try:
        df_routes = client.query(sql_routes).to_dataframe()
        print(df_routes.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

    # 3. FLUGHAFEN (Die Cash Cows)
    print("\n✈️ FLUGHAFEN STATS (agg_airport_trips):")
    sql_air = f"""
    SELECT year, trip_category, source_system, total_trips, avg_ticket
    FROM `{PROJECT_ID}.{AGG_DATASET}.agg_airport_trips`
    ORDER BY total_trips DESC
    LIMIT 5
    """
    try:
        df_air = client.query(sql_air).to_dataframe()
        print(df_air.to_string(index=False))
    except Exception as e:
        print(f"Fehler: {e}")

check_new_marts()

--- 📊 FINAL DATA MART CHECK ---

🕐 TOP 5 ZEITFENSTER (agg_time_trends):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 day_name  hour_of_day source_system  trip_count  avg_fare
 Thursday           18        YELLOW      429644     29.42
Wednesday           18        YELLOW      423982     28.90
  Tuesday           18        YELLOW      403973     28.57
   Friday           18        YELLOW      397860     28.56
 Thursday           17        YELLOW      396463     31.39

📍 TOP 5 ROUTEN (agg_route_stats):


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


pickup_borough dropoff_borough  trip_count  avg_cost
     Manhattan       Manhattan    20908052     20.34
     Manhattan       Manhattan     7358395     12.41
     Manhattan       Manhattan     6957972     12.28
     Manhattan       Manhattan     6223453     12.24
     Manhattan       Manhattan     5457148     12.20

✈️ FLUGHAFEN STATS (agg_airport_trips):
 year                 trip_category source_system  total_trips  avg_ticket
 2015 Standard Rate to Airport Zone        YELLOW       587996       43.19
 2016 Standard Rate to Airport Zone        YELLOW       557057       43.96
 2017 Standard Rate to Airport Zone        YELLOW       489819       45.04
 2018 Standard Rate to Airport Zone        YELLOW       451924       44.57
 2019 Standard Rate to Airport Zone        YELLOW       379443       44.72


/Users/lania/Documents/GitHub/bi_project_task2/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [3]:
# Funktion für 1) Peak Hours – Taxi Demand
def create_agg_peak_hours():
    """
    Erstellt die Tabelle 'agg_peak_hours' im Aggregational Layer.
    
    Logik:
    1. Basis: dimensional.Fact_Trips (Hier sind die Zeitstempel und die Fahrten selbst)
    2. Join: dimensional.dim_location (Um statt LocationIDs echte Borough-Namen zu haben)
    3. Ergebnis: Eine kleine, schnelle Tabelle, die nur noch Stunden und Anzahl enthält.
    """
    
    # Ziel-Tabelle im Aggregational Dataset
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_peak_hours"
    
    print(f"Erstelle Tabelle für Peak Hours: {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        -- 1. Zeit aus der Faktentabelle extrahieren
        EXTRACT(HOUR FROM f.pickup_datetime) as hour,
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        
        -- 2. Dimensionen aus Fact (Taxi Typ) und Dim (Borough)
        f.source_system as taxi_type,
        loc.Borough as borough,
        
        -- 3. Die eigentliche Messgröße (Anzahl Fahrten)
        COUNT(f.trip_id) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    -- Hier verbinden wir das Star Schema: Fakt + Dimension
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
    
    -- Aggregation auf die Ebene, die das Dashboard braucht
    GROUP BY 
        hour, 
        year, 
        taxi_type, 
        borough
    """
    
    try:
        # Query ausführen
        job = client.query(sql)
        job.result()
        print(f"Tabelle '{table_id}' erfolgreich erstellt!")
        
        # Kurzer Check
        rows = client.query(f"SELECT COUNT(*) as cnt FROM `{table_id}`").to_dataframe()
        print(f"   -> Enthält {rows['cnt'][0]} aggregierte Zeilen.")
        
    except Exception as e:
        print(f"Fehler: {e}")

# Ausführen
create_agg_peak_hours()

Erstelle Tabelle für Peak Hours: taxi-bi-project.aggregational.agg_peak_hours ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_peak_hours' erfolgreich erstellt!
   -> Enthält 5979 aggregierte Zeilen.


/Users/vincentgoldmann/Documents/GitHub/bi_project_task2/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
# Funktion für 2) Fare Distribution – Boxplot Stats
def create_agg_fare_stats():
    """
    Erstellt die Tabelle 'agg_fare_stats' im Aggregational Layer.
    
    Zweck: 
    Bereitstellung von statistischen Daten für Boxplots (Preisverteilung).
    Anstatt Rohdaten zu laden, berechnen wir Quantile (Min, 25%, Median, 75%, Max).
    
    Logik:
    1. Basis: dimensional.Fact_Trips (für fare_amount)
    2. Join: dimensional.dim_location (für Boroughs)
    3. Berechnung: APPROX_QUANTILES teilt die Daten in 100 Teile.
    """
    
    # Ziel-Tabelle
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_fare_stats"
    
    print(f"🔨 Erstelle Tabelle für Preis-Statistiken: {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        -- 1. Dimensionen für Filter
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        loc.Borough as borough,
        
        -- 2. Statistische Kennzahlen für den Boxplot (BigQuery Magic!)
        -- Wir berechnen hier die "Box" und die "Whiskers" vor.
        
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(0)] as min_fare,
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(25)] as q1_fare,     -- Unteres Quartil (25%)
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(50)] as median_fare, -- Der Median (50%)
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(75)] as q3_fare,     -- Oberes Quartil (75%)
        APPROX_QUANTILES(f.fare_amount, 100)[OFFSET(95)] as max_fare,    -- 95% Perzentil (statt echtem Max, um extreme Ausreißer auszublenden)
        
        -- Anzahl der Fahrten zur Einordnung
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.fare_amount > 0          -- Nur gültige Fahrten
      AND f.fare_amount < 1000       -- Technischer Filter: Unrealistische Werte (>1000$) ignorieren
      AND loc.Borough != 'Unknown'   
      AND loc.Borough != 'NV'
      
    GROUP BY 
        year, 
        taxi_type, 
        borough
    """
    
    try:
        # Query ausführen
        client.query(sql).result()
        print(f"Tabelle '{table_id}' erfolgreich erstellt!")
        
        # Vorschau
        print("Vorschau (Median Preise):")
        df_preview = client.query(f"SELECT * FROM `{table_id}` LIMIT 5").to_dataframe()
        print(df_preview.to_string(index=False))
        
    except Exception as e:
        print(f"Fehler: {e}")

# Ausführen
create_agg_fare_stats()

🔨 Erstelle Tabelle für Preis-Statistiken: taxi-bi-project.aggregational.agg_fare_stats ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_fare_stats' erfolgreich erstellt!
Vorschau (Median Preise):
 year taxi_type borough  min_fare  q1_fare  median_fare  q3_fare  max_fare  trip_count
 2010    YELLOW   Bronx      2.50      5.7          8.1     12.9      28.9       15879
 2011    YELLOW   Bronx      2.50      5.7          8.1     12.9      27.7       21179
 2012    YELLOW   Bronx      2.50      5.7          8.5     13.7      28.9       16982
 2015     GREEN   Bronx      0.01      6.5          9.5     14.5      29.5     1203056
 2016     GREEN   Bronx      0.01      7.0         10.0     15.0      31.0      762069


In [6]:
def create_agg_tip_stats():
    # Erstellt die Tabelle für die Trinkgeld-Analyse.
    # Berücksichtigt nur Kreditkartenzahlungen, da Barzahlungen keine Tip-Informationen enthalten.
    
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_tip_stats"
    print(f"Erstelle Tabelle: {table_id}")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        loc.Borough as borough,
        
        -- Speichern der Summen für spätere Berechnung des gewichteten Durchschnitts
        SUM(f.tip_amount) as total_tip,
        SUM(f.fare_amount) as total_fare,
        COUNT(*) as card_trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
    JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_payment_type` pay 
        ON f.payment_type_id = pay.payment_type_id
        
    WHERE pay.payment_description = 'Credit Card' 
      AND f.fare_amount > 0
      AND loc.Borough NOT IN ('Unknown', 'NV')
      
    GROUP BY year, taxi_type, borough
    """
    
    try:
        client.query(sql).result()
        print(f"Tabelle '{table_id}' erstellt.")
    except Exception as e:
        print(f"Fehler: {e}")

create_agg_tip_stats()

Erstelle Tabelle: taxi-bi-project.aggregational.agg_tip_stats
Tabelle 'taxi-bi-project.aggregational.agg_tip_stats' erstellt.


In [8]:
# Funktion für 4) Demand Shift over Years
def create_agg_demand_years():
    """
    Erstellt 'agg_demand_years' im Aggregational Layer.
    """
    
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_demand_years"
    
    print(f"🔨 Erstelle Tabelle für Jahres-Trends (Fix): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        
        -- Wenn kein Borough gefunden wird (NULL), nennen wir es 'Unknown'
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        COUNT(*) as total_trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
     -- LEFT JOIN, damit Fahrten ohne Location erhalten bleiben
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      
    GROUP BY 
        year, 
        taxi_type, 
        borough
    """
    
    try:
        client.query(sql).result()
        print(f"Tabelle '{table_id}' erfolgreich aktualisiert!")
        
        check_sql = f"""
            SELECT sum(total_trips) as cnt 
            FROM `{table_id}` 
            WHERE year=2016 AND taxi_type='FHV'
        """
        cnt = client.query(check_sql).to_dataframe()['cnt'][0]
        print(f"   -> FHV Trips in 2016: {cnt:,.0f}")
        
    except Exception as e:
        print(f"Fehler: {e}")

# Ausführen
create_agg_demand_years()

🔨 Erstelle Tabelle für Jahres-Trends (Fix): taxi-bi-project.aggregational.agg_demand_years ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_demand_years' erfolgreich aktualisiert!
   -> FHV Trips in 2016: 1,080 (Sollte jetzt Millionen sein)


/Users/vincentgoldmann/Documents/GitHub/bi_project_task2/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [12]:
def create_agg_weekly_patterns():
    """
    Erstellt 'agg_weekly_patterns'.
    FIX: Nutzt 'day_of_week_num' aus dim_date.
    """
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_weekly_patterns"
    print(f"🔨 Erstelle Wochentags-Tabelle (Final Fix): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        -- Wochentag-Name (z.B. 'Monday')
        d.day_name,
        
        -- Wochentag-Nummer für die Sortierung (aus deiner dim_date)
        -- Wir nennen es 'day_of_week', damit der Data-Access-Code es sofort findet
        d.day_of_week_num as day_of_week,
        
        EXTRACT(HOUR FROM f.pickup_datetime) as hour,
        
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    INNER JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_date` d
        ON DATE(f.pickup_datetime) = d.date_key
        
    WHERE f.pickup_datetime IS NOT NULL
      
    GROUP BY 
        year, taxi_type, borough, d.day_name, d.day_of_week_num, hour
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich erstellt!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_weekly_patterns()

🔨 Erstelle Wochentags-Tabelle (mit dim_date): taxi-bi-project.aggregational.agg_weekly_patterns ...
Fehler: 400 Name day_of_week not found inside d at [10:11]; reason: invalidQuery, location: query, message: Name day_of_week not found inside d at [10:11]

Location: EU
Job ID: dcd820a8-dcad-409b-a922-4fa0e6bc3e37



In [14]:
def create_agg_fare_dist():
    """
    Erstellt 'agg_fare_dist'.
    Aggregiert Daten in Cluster (Bins) für Distanz und Preis.
    """
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_fare_dist"
    print(f"Erstelle Aggregations-Tabelle (Fare/Dist): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        COALESCE(loc.Borough, 'Unknown') as borough,
        
        -- Cluster-Bildung: Distanz auf 0.2 Meilen, Preis auf 1 Dollar gerundet
        ROUND(f.trip_distance * 5) / 5 as dist_bin,
        ROUND(f.fare_amount, 0) as fare_bin,
        
        -- Anzahl der Fahrten pro Cluster
        COUNT(*) as trip_count
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc 
        ON f.pickup_location_id = loc.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      AND f.trip_distance > 0 AND f.trip_distance < 100
      AND f.fare_amount > 0 AND f.fare_amount < 500
      
    GROUP BY 
        year, taxi_type, borough, dist_bin, fare_bin
    """
    
    try:
        client.query(sql).result()
        print(f"Tabelle '{table_id}' erfolgreich erstellt!")
    except Exception as e:
        print(f"Fehler: {e}")

create_agg_fare_dist()

🔨 Erstelle Aggregations-Tabelle (Fare/Dist): taxi-bi-project.aggregational.agg_fare_dist ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_fare_dist' erfolgreich erstellt!


In [15]:
def create_agg_flows():
    """
    Erstellt 'agg_borough_flows'.
    Basis für das Flow-Diagramm (Pickup -> Dropoff).
    """
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_borough_flows"
    print(f"Erstelle Flow-Tabelle: {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    SELECT
        EXTRACT(YEAR FROM f.pickup_datetime) as year,
        f.source_system as taxi_type,
        
        -- Wir bereinigen NULL-Werte, damit im Diagramm keine Lücken entstehen
        COALESCE(loc_pu.Borough, 'Unknown') as pickup_borough,
        COALESCE(loc_do.Borough, 'Unknown') as dropoff_borough,
        
        COUNT(*) as trips
        
    FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_pu 
        ON f.pickup_location_id = loc_pu.location_id
    LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc_do 
        ON f.dropoff_location_id = loc_do.location_id
        
    WHERE f.pickup_datetime IS NOT NULL
      -- Wir schließen technisch bedingte 'Unknown'-Zonen aus, falls gewünscht.
      -- Für eine saubere Optik oft besser:
      AND loc_pu.Borough IS NOT NULL AND loc_pu.Borough != 'Unknown'
      AND loc_do.Borough IS NOT NULL AND loc_do.Borough != 'Unknown'
      
    GROUP BY 1, 2, 3, 4
    """
    
    try:
        client.query(sql).result()
        print(f"Tabelle '{table_id}' erfolgreich erstellt!")
    except Exception as e:
        print(f"Fehler: {e}")

create_agg_flows()

Erstelle Flow-Tabelle: taxi-bi-project.aggregational.agg_borough_flows ...
Tabelle 'taxi-bi-project.aggregational.agg_borough_flows' erfolgreich erstellt!


In [18]:
def create_agg_revenue_efficiency():
    """
    Erstellt 'agg_revenue_efficiency'.
    
    Features:
    - Kategorisierung nach Fahrtdauer (Kurz/Mittel/Lang) -> für die Story
    - Inklusive BOROUGH -> damit der Dashboard-Filter funktioniert!
    """
    table_id = f"{PROJECT_ID}.{AGG_DATASET}.agg_revenue_efficiency"
    print(f"🔨 Erstelle Efficiency-Tabelle (Duration Buckets + Borough): {table_id} ...")
    
    sql = f"""
    CREATE OR REPLACE TABLE `{table_id}` AS
    WITH raw_calc AS (
        SELECT
            EXTRACT(YEAR FROM f.pickup_datetime) as year,
            f.source_system as taxi_type,
            
            -- WICHTIG: Borough für den Filter holen
            COALESCE(loc.Borough, 'Unknown') as borough,
            
            -- Dauer berechnen
            TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, MINUTE) as duration_min,
            f.fare_amount
            
        FROM `{PROJECT_ID}.{DIM_DATASET}.Fact_Trips` f
        LEFT JOIN `{PROJECT_ID}.{DIM_DATASET}.dim_location` loc
            ON f.pickup_location_id = loc.location_id
            
        WHERE f.pickup_datetime IS NOT NULL
          AND f.fare_amount > 0
          -- Plausible Dauer: 1 bis 180 Minuten
          AND TIMESTAMP_DIFF(f.dropoff_datetime, f.pickup_datetime, MINUTE) BETWEEN 1 AND 180
    ),
    
    categorized AS (
        SELECT 
            year,
            taxi_type,
            borough,
            duration_min,
            fare_amount,
            
            -- Die Kategorien für die x-Achse
            CASE
                WHEN duration_min < 10 THEN '1. Kurzstrecke (< 10 min)'
                WHEN duration_min < 20 THEN '2. Mittel (10 - 20 min)'
                WHEN duration_min < 45 THEN '3. Lang (20 - 45 min)'
                ELSE '4. Sehr Lang (> 45 min)'
            END as trip_category,
            
            SAFE_DIVIDE(fare_amount, duration_min) as fare_per_min
        FROM raw_calc
    )
    
    SELECT
        year,
        taxi_type,
        borough,       -- Jetzt dabei!
        trip_category, -- Unsere x-Achse
        
        COUNT(*) as total_trips,
        APPROX_QUANTILES(fare_per_min, 4) as quantiles
        
    FROM categorized
    GROUP BY 1, 2, 3, 4
    """
    
    try:
        client.query(sql).result()
        print(f"✅ Tabelle '{table_id}' erfolgreich erstellt (mit Borough & Duration)!")
    except Exception as e:
        print(f"❌ Fehler: {e}")

create_agg_revenue_efficiency()

🔨 Erstelle Efficiency-Tabelle (Boxplot Stats): taxi-bi-project.aggregational.agg_revenue_efficiency ...
✅ Tabelle 'taxi-bi-project.aggregational.agg_revenue_efficiency' erfolgreich erstellt!
